In [1]:
import numpy as np
import pandas as pd
import tweepy
import json
from tweepy import OAuthHandler

In [2]:
consumer_key = "LzsCOl12ZZ79FObx7BV9a1R3a"
consumer_secret = "gODxZ1kHJHDGgYfyP12AUN37UTlDd6yn3OsSV9Tl0iwu0K0cRY"
access_token = "296102538-kYIK8SULCmRK4vgvYD2DhKbwvSXrgTdTkWh0Nvsp"
access_token_secret = "Ib9obwgcE5tZk4S6tY5rAqI5iXOeJc9FGFg28qHoa4YpP"

In [3]:
auth = tweepy.OAuthHandler( consumer_key , consumer_secret )
auth.set_access_token( access_token , access_token_secret )
api = tweepy.API(auth)

In [4]:
query = "RCB winning IPL"

In [5]:
Tweets = api.search( query , count=10 , lang='en' , exclude='retweets' , tweet_mode='extended' )
# tweet_mode='extended'
# tweet_mode='compat'

for tweet in Tweets:
    print(tweet,"\n")
    print(".....................................\n")

Status(_api=<tweepy.api.API object at 0x7ff883ad9250>, _json={'created_at': 'Wed Jan 12 16:23:48 +0000 2022', 'id': 1481300890372161536, 'id_str': '1481300890372161536', 'full_text': "@Newlands_79 @SunRisers Having an IPL Cup on their name winning Over RCB if that's a small team then the franchise who lost to them is even lesser", 'truncated': False, 'display_text_range': [24, 146], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'Newlands_79', 'name': 'C/``\uea00¹⁸', 'id': 1474664342268235777, 'id_str': '1474664342268235777', 'indices': [0, 12]}, {'screen_name': 'SunRisers', 'name': 'SunRisers Hyderabad', 'id': 989137039, 'id_str': '989137039', 'indices': [13, 23]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': 1480433660687912960, 'in_reply_to_status_id_str': '1480433660687912960', 'in_reply_to_user_id': 14746643

In [6]:
df = pd.DataFrame(columns = ['Tweets' , 'User' , 'User_statuses_count' , 
                            'user_followers' , 'User_location' , 'User_verified' ,
                            'fav_count' , 'rt_count' , 'tweet_date'] )
print(df)
# print(df.shape)

Empty DataFrame
Columns: [Tweets, User, User_statuses_count, user_followers, User_location, User_verified, fav_count, rt_count, tweet_date]
Index: []


In [7]:
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'User_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i = i+1
        if i == 1000:
            break
        else:
            pass

In [8]:
stream(data=["RCB winning IPL"] , file_name='my_tweets')

In [9]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date
0,@Newlands_79 @SunRisers Having an IPL Cup on t...,Yagnik vicky,155,8,,False,0,0,2022-01-12 16:23:48
1,@msd_ntr79 RCB is cursed bcoz of Vijay Maliya ...,Rajesh,12744,223,,False,0,1,2022-01-12 10:57:05
2,RT @teamsaver: @mufaddal_vohra Ratio. RCB winn...,Gagandeep M,48111,82,"Karnataka, India",False,0,1,2022-01-12 09:24:03
3,RT @crictallkk: Imagine RCB winning TATA IPL 2...,Subhash,861,2,,False,0,1,2022-01-12 08:16:17
4,Imagine RCB winning TATA IPL 2022 ✨\n\n#IPLAuc...,Alfred,2820,45,Bird App,False,3,1,2022-01-12 08:02:12


In [10]:
from textblob import TextBlob

In [11]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [12]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [13]:
df['clean_tweet'] = df['Tweets'].apply(lambda x : clean_tweet(x))

In [14]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet
0,@Newlands_79 @SunRisers Having an IPL Cup on t...,Yagnik vicky,155,8,,False,0,0,2022-01-12 16:23:48,79 Having an IPL Cup on their name winning Ove...
1,@msd_ntr79 RCB is cursed bcoz of Vijay Maliya ...,Rajesh,12744,223,,False,0,1,2022-01-12 10:57:05,ntr79 RCB is cursed bcoz of Vijay Maliya looti...
2,RT @teamsaver: @mufaddal_vohra Ratio. RCB winn...,Gagandeep M,48111,82,"Karnataka, India",False,0,1,2022-01-12 09:24:03,RT vohra Ratio RCB winning IPL next season
3,RT @crictallkk: Imagine RCB winning TATA IPL 2...,Subhash,861,2,,False,0,1,2022-01-12 08:16:17,RT Imagine RCB winning TATA IPL 2022 IPLAuctio...
4,Imagine RCB winning TATA IPL 2022 ✨\n\n#IPLAuc...,Alfred,2820,45,Bird App,False,3,1,2022-01-12 08:02:12,Imagine RCB winning TATA IPL 2022 IPLAuction20...


In [15]:
df['Sentiment'] = df['clean_tweet'].apply(lambda x : analyze_sentiment(x) )
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
0,@Newlands_79 @SunRisers Having an IPL Cup on t...,Yagnik vicky,155,8,,False,0,0,2022-01-12 16:23:48,79 Having an IPL Cup on their name winning Ove...,Positive
1,@msd_ntr79 RCB is cursed bcoz of Vijay Maliya ...,Rajesh,12744,223,,False,0,1,2022-01-12 10:57:05,ntr79 RCB is cursed bcoz of Vijay Maliya looti...,Positive
2,RT @teamsaver: @mufaddal_vohra Ratio. RCB winn...,Gagandeep M,48111,82,"Karnataka, India",False,0,1,2022-01-12 09:24:03,RT vohra Ratio RCB winning IPL next season,Positive
3,RT @crictallkk: Imagine RCB winning TATA IPL 2...,Subhash,861,2,,False,0,1,2022-01-12 08:16:17,RT Imagine RCB winning TATA IPL 2022 IPLAuctio...,Positive
4,Imagine RCB winning TATA IPL 2022 ✨\n\n#IPLAuc...,Alfred,2820,45,Bird App,False,3,1,2022-01-12 08:02:12,Imagine RCB winning TATA IPL 2022 IPLAuction20...,Positive


In [16]:
n = 1
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 @msd_ntr79 RCB is cursed bcoz of Vijay Maliya looting don't bring IPL here winning in Aus, Eng is not cursed we hav… https://t.co/CSbcOeHXKx

Clean tweet:
 ntr79 RCB is cursed bcoz of Vijay Maliya looting don t bring IPL here winning in Aus Eng is not cursed we hav

Sentiment of the tweet:
 Positive


In [17]:
n = 20
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 @vd12op I am saying anybody doesn't interested in MI even after winning 5 trophies. But all are interested in RCB e… https://t.co/wtMHH11xHA

Clean tweet:
 I am saying anybody doesn t interested in MI even after winning 5 trophies But all are interested in RCB e

Sentiment of the tweet:
 Positive


In [20]:
n = 10
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RCB not winning these TATA IPL either.😭😭😭

Clean tweet:
 RCB not winning these TATA IPL either

Sentiment of the tweet:
 Negative


In [21]:
df[df.Sentiment == 'Positive'].shape[0]

25

In [22]:
df[df.Sentiment == 'Neutral'].shape[0]

1

In [23]:
df[df.Sentiment == 'Negative'].shape[0]

5